In [ ]:
import pandas as pd
import time
import datetime
import math
import ims
import utils
import locale

# Analyse segment statistics and generate an HTML table for public consumption

In [ ]:
# gather data
md = utils.get_segment_metadata()
rl_ = utils.get_ridelogs()

# save a table aside
md_meta = md[['id', 'name', 'distance', 'region_name', 'region_url', 'closest_ims']].copy()

In [ ]:
d5 = rl_.copy()

# add the closest IMS station
d6 = d5.merge(md_meta, how='right', left_on=['segment_id'], right_on=['id'])

In [ ]:
rain_days = utils.get_rain_days(d6)

# Add rain measurements
d7 = d6.merge(rain_days, how='left', left_on=['closest_ims', 'date'], right_on=['closest_ims', 'date'])

# cumulative measures of rainfall

d7.sort_values('date', inplace=True)
d7['rain_7d'] = d7.groupby('segment_id')['rain_mm'].apply(lambda x : x.rolling(7).sum()).fillna(0)
#data['soil_moisture'] = data.groupby('segment_id')['rain_mm'].apply(utils.bathtub)
df_orig = d7.copy()

In [ ]:
# trim to just most recent day
lastdate = df_orig['date'].max()
df = df_orig.query("date == @lastdate").copy()


In [ ]:
if False:
    from matplotlib.dates import DateFormatter
    sns.lineplot(data=d8, dashes=False, marker='o')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

    plt.gca().xaxis.set_major_formatter(DateFormatter("%m-%d"))
    plt.xticks(rotation=45)
    None

In [ ]:
# prepare for display as nice HTML
def link2(a, id):
    return f'<a href="{a}">{id}</a>'

rideability_color = lambda x: '<div style="background-color: {}">{}</div>'.format(('Chartreuse' if x>80 else 'DarkOrange' if x>30 else 'OrangeRed'), x)


In [ ]:
dfout = df.copy()

#format the date
dateout = lastdate.strftime('%d/%m/%Y')

locale.setlocale(locale.LC_ALL, 'he_IL')
weekday_name = lastdate.strftime('%A')

dfout['link'] = dfout.apply(lambda x: link2(f"https://www.strava.com/segments/{x['id']}", x['name']), axis=1)
dfout['region_link'] = dfout.apply(lambda x: link2(x['region_url'], x['region_name']), axis=1)
dfout['distance'] = dfout['distance'].map(lambda x : "%.0f" % x)

dfout.drop(columns=['date', 'name', 'id', 'distance'], inplace=True)
dfout['nrides'] = dfout['nrides'].map(lambda x : "" if math.isnan(x) else "%.0f%%" % (100*x))
dfout['rain_mm'] = dfout['rain_mm'].map(lambda x : "%.1f" % x)
dfout['rain_7d'] = dfout['rain_7d'].map(lambda x : "%.0f" % x)
                                        
# re-order columns
dfout = dfout[['nrides', 'link', 'region_link', 'rain_mm', 'rain_7d']].copy()

nrides_str = "מספר רכיבות היום ביחס ליום %s ממוצע" % (weekday_name)

print(nrides_str)

dfout.rename(columns = {'link' : 'מקטע', 'region_link' : 'איזור',
                        'nrides' : nrides_str,
                        'rain_mm' : 'גשם יומי מ״מ', 'rain_7d' : 'גשם מצטבר שבועי מ״מ'},
             inplace=True)

htmlout = dfout.to_html(#formatters={nrides_str: rideability_color},
                        render_links=True, classes="table",
                        escape=False, index=False, border=1)


# Add decorations and save to file

title = 'מדד בוציות בסינגלים'
update_ts = '<br>' +  "עדכון אחרון: {}".format(dateout) + '</br>\n'

with open('preamble.txt') as f:
    preamble = " ".join([l.rstrip() for l in f]) 

with open('epilog.txt') as f:
    epilog = "\n".join([l.rstrip() for l in f]) 

html_preamble = '<html><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1">\n<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/css/bootstrap.min.css">\n<title>' + title + '</title>\n</head><body dir=rtl>\n' + preamble + "\n" + update_ts + '<div class="container">\n'
htmlout = html_preamble + htmlout + "</div>\n" + epilog

fileout = "data/out/rides.html"

with open(fileout, "w", encoding="utf-8") as file:
    file.write(htmlout)